In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold, cross_validate, LeaveOneOut
from torch.utils.data import Dataset, DataLoader
from time import sleep
import numpy as np
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon # for maybe JSD
from MatrixVectorizer import MatrixVectorizer
import networkx as nx # for maybe centrality measures
import random
from torch.nn import Linear, ReLU, Sequential, Flatten, Unflatten, ModuleList, Sigmoid, LeakyReLU
import matplotlib.pyplot as plt
import csv
import time
import psutil
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import community.community_louvain as community_louvain
import os


In [ ]:
def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")
    adj_matrix = np.array(adj_matrix)
    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict


def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path='ID-randomCV.csv'):
    print(true_hr_matrices.shape)

    # print(predicted_hr_matrices.shape)
    
    num_subjects = true_hr_matrices.shape[0]
    print("numsubjects matrix", num_subjects)
    print("pred", len(predicted_hr_matrices))
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i]
        pred_matrix = predicted_hr_matrices[i]
        print("true matrix", true_matrix.shape)
        print("pred", pred_matrix.shape)

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")


In [ ]:
# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

# Dataset

## Anti-Vectorization

In [ ]:
# Vectorization
def vectorize_symmetric_matrix(matrix):
    """Vectorize a symmetric matrix"""
    vectorizer = MatrixVectorizer()
    return vectorizer.vectorize(matrix)

def devectorize_symmetric_matrix(vector, size):
    """Devectorize a symmetric matrix"""
    vectorizer = MatrixVectorizer()
    return vectorizer.anti_vectorize(vector, size)

In [ ]:
# Load the data
lr_train1 = pd.read_csv('../Cluster-CV/Fold1/lr_clusterA.csv').values
hr_train1 = pd.read_csv('../Cluster-CV/Fold1/hr_clusterA.csv').values

lr_train2 = pd.read_csv('../Cluster-CV/Fold2/lr_clusterB.csv').values
hr_train2 = pd.read_csv('../Cluster-CV/Fold2/hr_clusterB.csv').values

lr_train3 = pd.read_csv('../Cluster-CV/Fold3/lr_clusterC.csv').values
hr_train3 = pd.read_csv('../Cluster-CV/Fold3/hr_clusterC.csv').values

In [ ]:
lr_train_anti_vectorized1 = torch.tensor([devectorize_symmetric_matrix(x, 160) for x in lr_train1])
hr_train_anti_vectorized1 = torch.tensor([devectorize_symmetric_matrix(x, 268) for x in hr_train1])

lr_train_anti_vectorized2 = torch.tensor([devectorize_symmetric_matrix(x, 160) for x in lr_train2])
hr_train_anti_vectorized2 = torch.tensor([devectorize_symmetric_matrix(x, 268) for x in hr_train2])

lr_train_anti_vectorized3 = torch.tensor([devectorize_symmetric_matrix(x, 160) for x in lr_train3])
hr_train_anti_vectorized3 = torch.tensor([devectorize_symmetric_matrix(x, 268) for x in hr_train3])


In [ ]:
print(len(lr_train_anti_vectorized1))
print(len(hr_train_anti_vectorized1))
print(lr_train_anti_vectorized1.shape)
print(len(lr_train_anti_vectorized2))
print(len(hr_train_anti_vectorized2))
print(lr_train_anti_vectorized2.shape)
print(len(lr_train_anti_vectorized3))
print(len(hr_train_anti_vectorized3))
print(lr_train_anti_vectorized3.shape)



## Create dataset

In [ ]:
def pad_graph_to_expected_size(matrix, expected_size=160):
    # Ensure the matrix is on CPU for manipulation
    matrix_np = matrix.detach().cpu().numpy() if isinstance(matrix, torch.Tensor) else matrix
    current_size = matrix_np.shape[0]
    
    if current_size < expected_size:
        padding_size = expected_size - current_size
        padded_matrix_np = np.pad(matrix_np, ((0, padding_size), (0, padding_size)), mode='constant', constant_values=0)
        padded_matrix = torch.tensor(padded_matrix_np, dtype=torch.float).to(device)
    else:
        padded_matrix = torch.tensor(matrix_np, dtype=torch.float).to(device) if not isinstance(matrix, torch.Tensor) else matrix
    
    return padded_matrix

# Dataset
class BrainConnectivityDataset(Dataset):
    def __init__(self, features, labels, augment=False, augmentation_methods=None):
        self.features = features.clone().detach().float().to(device)
        if labels is not None:
            self.labels = labels.clone().detach().float().to(device)
        else: 
            self.labels = None
        self.augment = augment
        self.augmentation_methods = augmentation_methods

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        if self.labels is not None:
            label = self.labels[idx] 
        else:
            label = None

        # Apply data augmentation if enabled
        if self.augment and self.augmentation_methods is not None:
            for augment in self.augmentation_methods:
                feature = augment(feature)

        feature = pad_graph_to_expected_size(feature, expected_size=160)

        return feature, label


# Data Augmentation Methods
def node_dropping(matrix, drop_rate=0.2):
    """
    Randomly drops nodes (and their corresponding connections) from the graph.
    drop_rate: Proportion of nodes to drop.
    """
    # Convert to numpy array for manipulation if it's a torch tensor
    if isinstance(matrix, torch.Tensor):
        matrix = matrix.cpu().numpy()
    
    num_nodes = matrix.shape[0]
    num_drop = int(num_nodes * drop_rate)
    drop_indices = np.random.choice(num_nodes, num_drop, replace=False)
    
    # Create a mask to keep the nodes not in drop_indices
    keep_mask = np.ones(num_nodes, dtype=bool)
    keep_mask[drop_indices] = False

    # Filter the matrix to keep only the remaining nodes and connections
    matrix_filtered = matrix[np.ix_(keep_mask, keep_mask)]

    return torch.tensor(matrix_filtered, dtype=torch.float).to(device)

def edge_perturbation(matrix, perturb_rate=0.05):
    """
    Randomly perturbs edges in the graph by adding or removing them.
    perturb_rate: Proportion of edges to perturb.
    """
    if isinstance(matrix, torch.Tensor):
        matrix = matrix.cpu().numpy()

    num_edges = int(np.triu(matrix, 1).sum())
    num_perturb = int(num_edges * perturb_rate)

    # Get upper triangular indices excluding the diagonal
    triu_indices = np.triu_indices(matrix.shape[0], k=1)
    
    for _ in range(num_perturb):
        edge_index = np.random.randint(len(triu_indices[0]))
        i, j = triu_indices[0][edge_index], triu_indices[1][edge_index]
        # Flip the edge state
        matrix[i, j] = matrix[j, i] = 1 - matrix[i, j]
        
    if isinstance(matrix, torch.Tensor):
        matrix = torch.tensor(matrix_np, dtype=torch.float).to(device)

    return matrix

def subgraph_sampling(matrix, sample_size=0.8):
    """
    Samples a subgraph from the original graph by randomly selecting a subset of nodes.
    sample_size: Proportion of nodes to include in the subgraph.
    """
    if isinstance(matrix, torch.Tensor):
        matrix_np = matrix.cpu().numpy()
    else:
        matrix_np = matrix

    num_nodes = matrix.shape[0]
    sample_num = int(num_nodes * sample_size)
    sampled_indices = np.random.choice(num_nodes, sample_num, replace=False)
    
    matrix_sampled = matrix_np[np.ix_(sampled_indices, sampled_indices)]

    if isinstance(matrix, torch.Tensor):
        return torch.tensor(matrix_sampled, dtype=torch.float).to(device)
    else:
        return matrix_sampled

def feature_noising(matrix, noise_level=0.01):
    """
    Adds Gaussian noise to the features (edge weights) of the graph.
    noise_level: Standard deviation of the Gaussian noise to add.
    """
    if isinstance(matrix, torch.Tensor):
        noise = torch.normal(0, noise_level, size=matrix.size(), device=matrix.device)
        matrix_noised = matrix + noise
        matrix_noised.fill_diagonal_(0) 
        matrix_noised = torch.clamp(matrix_noised, 0, 1)  # Clamp values to ensure they're within [0, 1]
        return matrix_noised
    else:
        noise = np.random.normal(0, noise_level, matrix.shape)
        matrix_noised = matrix + noise
        np.fill_diagonal(matrix_noised, 0)
        matrix_noised = np.clip(matrix_noised, 0, 1)  # Clip values for numpy arrays
        return torch.tensor(matrix_noised, dtype=torch.float).to(device)

augmentation_methods = [
    lambda x: node_dropping(x, drop_rate=0.2),
    lambda x: edge_perturbation(x, perturb_rate=0.05),
    lambda x: subgraph_sampling(x, sample_size=0.8),
    lambda x: feature_noising(x, noise_level=0.01)
]

In [ ]:
def compute_centrality_metrics(pred_matrices, gt_matrices):
    mae_bc = []
    mae_ec = []
    mae_pc = []
    pred_1d_list = []
    gt_1d_list = []

    # Iterate over each test sample
    for i in range(len(pred_matrices)):
        # Convert adjacency matrices to NetworkX graphs
        pred_graph = nx.from_numpy_array(pred_matrices[i], edge_attr="weight")
        gt_graph = nx.from_numpy_array(gt_matrices[i], edge_attr="weight")

        # Compute centrality measures
        pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
        pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
        pred_pc = nx.pagerank(pred_graph, weight="weight")

        gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
        gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
        gt_pc = nx.pagerank(gt_graph, weight="weight")

        # Convert centrality dictionaries to lists
        pred_bc_values = list(pred_bc.values())
        pred_ec_values = list(pred_ec.values())
        pred_pc_values = list(pred_pc.values())

        gt_bc_values = list(gt_bc.values())
        gt_ec_values = list(gt_ec.values())
        gt_pc_values = list(gt_pc.values())

        # Compute MAEs
        mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
        mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
        mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

        # Vectorize matrices
        pred_1d_list.append(MatrixVectorizer.vectorize(pred_matrices[i]))
        gt_1d_list.append(MatrixVectorizer.vectorize(gt_matrices[i]))

    # Compute average MAEs
    avg_mae_bc = sum(mae_bc) / len(mae_bc)
    avg_mae_ec = sum(mae_ec) / len(mae_ec)
    avg_mae_pc = sum(mae_pc) / len(mae_pc)

    # Concatenate flattened matrices
    pred_1d = np.concatenate(pred_1d_list)
    gt_1d = np.concatenate(gt_1d_list)

    # Compute metrics
    mae = mean_absolute_error(pred_1d, gt_1d)
    pcc = pearsonr(pred_1d, gt_1d)[0]
    js_dis = jensenshannon(pred_1d, gt_1d)

    return mae, pcc, js_dis, avg_mae_bc, avg_mae_ec, avg_mae_pc

In [ ]:
# 3-fold cross-validation
foldPred = None
def k_fold_cross_validation(get_model, loss_fn, evaluate, optimizer_fn, num_epochs, lr, step_size=10, gamma=0.1, training_only=True):
    losses = []
    all_metrics = {
        'mae': [],
        'pcc': [],
        'jsd': [],
        'avg_mae_bc': [],
        'avg_mae_ec': [],
        'avg_mae_pc': [],
        'ram_usage': [],
        'time': []
    }
    X = np.array([lr_train_anti_vectorized1, lr_train_anti_vectorized2, lr_train_anti_vectorized3], dtype="object")
    y = np.array([hr_train_anti_vectorized1, hr_train_anti_vectorized2, hr_train_anti_vectorized3], dtype="object")
    
    loo = LeaveOneOut()


    k = loo.get_n_splits(X)
    for i, (train_index, val_index) in enumerate(loo.split(X)):
        checkFold = i + 1
        print(f'Fold {i+1}/{k}')
        start_ram = psutil.Process().memory_info().rss / (1024 * 1024) # in MB
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        X_val_epoch = X_val[0][:20]
        y_val_epoch = y_val[0][:20] 
        
        # train_dataset = BrainConnectivityDataset(torch.from_numpy(X_train[0]), torch.from_numpy(y_train[0]), augment=True, augmentation_methods=augmentation_methods)
        # val_dataset = BrainConnectivityDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

        train_dataset = BrainConnectivityDataset(X_train[0], y_train[0], augment=True, augmentation_methods=augmentation_methods)
        val_dataset = BrainConnectivityDataset(X_val[0], y_val[0])

        # val_dataset_epoch = BrainConnectivityDataset(torch.from_numpy(X_val_epoch), torch.from_numpy(y_val_epoch))

        model = get_model()
        optimizer = optimizer_fn(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
        start_time = time.time()
        Final_epoch = 0
        breaks = False
        epochLosses = 0
        x = 0
        # to get number of epochs
        print("!!!!!!!GETTING NUMBER OF EPOCHS!!!!!!!")
        for epoch in range(num_epochs):
            if breaks == False:
                model.train()
                train_loss = 0
                for j in range(len(train_dataset)):
                    optimizer.zero_grad()
                    lr_graph, hr_graph = train_dataset[j]
                    loss = loss_fn(lr_graph, hr_graph, model)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                scheduler.step()
                print(f'epoch: {epoch+1}, loss: {train_loss/len(train_dataset)}')
                Final_epoch = epoch + 1
                # 0.00005
                if abs(epochLosses - loss.item()) < 0.00005:
                    x+= 1
                    print("x: ", x)
                    if x >= 10:
                        print("break")
                        breaks = True
                epochLosses = loss.item()
    
        # Actual training
        print("!!!!!!!!!ACTUAL TRAINING!!!!!!!!!")
        print("FINAL EPOCHS: ", Final_epoch)
        for epoch in range(Final_epoch):
            model.train()
            train_loss = 0
            for j in range(len(train_dataset)):
                optimizer.zero_grad()
                lr_graph, hr_graph = train_dataset[j]
                loss = loss_fn(lr_graph, hr_graph, model)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            scheduler.step()
            print(f'epoch: {epoch+1}, loss: {train_loss/len(train_dataset)}')
        
        final_ram = psutil.Process().memory_info().rss / (1024 * 1024) # in MB
        final_time = time.time()
        
        model.eval()
        with torch.no_grad():
            val_loss = 0
            pred_matrices = []
            gt_matrices = []
            for j in range(len(val_dataset)):
                lr_graph, hr_graph = val_dataset[j]
                hr_graph_pred, _ = model(lr_graph)
                val_loss += evaluate(hr_graph_pred, hr_graph).item()

                if not training_only:
                    pred_matrices.append(hr_graph_pred.cpu().numpy())
                    gt_matrices.append(hr_graph.cpu().numpy())

            val_loss /= len(val_dataset)
            print(f'val_loss: {val_loss}')
            losses.append(val_loss)

            if not training_only:
                mae, pcc, jsd, avg_mae_bc, avg_mae_ec, avg_mae_pc = compute_centrality_metrics(pred_matrices, gt_matrices)
                
                time_per_epoch = (final_time - start_time) / num_epochs
                ram_usage = final_ram - start_ram

                # Print metrics
                print("mae", mae)
                print("pcc", pcc)
                print("jsd", jsd)
                print("avg_mae_bc", avg_mae_bc)
                print("avg_mae_ec", avg_mae_ec)
                print("avg_mae_pc", avg_mae_pc)
                print(f'Training time per epoch: {time_per_epoch} seconds')
                print(f'Approximate RAM usage: {ram_usage} MB')

                all_metrics['mae'].append(mae)
                all_metrics['pcc'].append(pcc)
                all_metrics['jsd'].append(jsd)
                all_metrics['avg_mae_bc'].append(avg_mae_bc)
                all_metrics['avg_mae_ec'].append(avg_mae_ec)
                all_metrics['avg_mae_pc'].append(avg_mae_pc)
                all_metrics['ram_usage'].append(ram_usage)
                all_metrics['time'].append(time_per_epoch)              

            # Save predictions

            if checkFold == 1:
                foldPred = write_result_to_file(model, val_dataset, f'23-05predictions_fold_{i+1}!!.csv')
                evaluate_all(hr_train_anti_vectorized1, foldPred, "fold1CSV-Cluster.csv")
            elif checkFold == 2:
                foldPred = write_result_to_file(model, val_dataset, f'23-05predictions_fold_{i+1}!!.csv')
                evaluate_all(hr_train_anti_vectorized2, foldPred, "fold2CSV-Cluster.csv")
            else:
                foldPred = write_result_to_file(model, val_dataset, f'23-05predictions_fold_{i+1}!!.csv')
                evaluate_all(hr_train_anti_vectorized3, foldPred, "fold3CSV-Cluster.csv")
                

            
            del model
            del optimizer
            del scheduler
            del train_dataset
            del val_dataset
            torch.cuda.empty_cache()
            sleep(5)
            

    return losses, all_metrics

def write_result_to_file(model, test_dataset, filename='submission.csv'):
    graph_prediction = []
    # TODO: change to return all of the predictions 
    with open(filename, 'w') as file:
        file.write('ID,Predicted\n')
        model.eval()
        id = 1
        with torch.no_grad():
            for j in range(len(test_dataset)):
                lr_graph, _  = test_dataset[j]
                hr_graph_pred, _ = model(lr_graph)
                hr_graph_pred = hr_graph_pred.cpu().numpy()
                # hr_graph_pred = vectorize_symmetric_matrix(hr_graph_pred)
                graph_prediction.append(hr_graph_pred)
                for i, val in enumerate(hr_graph_pred):
                    file.write('x')
                    # graph_prediction.append((id, val, "Public"))
                    id += 1
    return graph_prediction

# Model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Linear, ReLU, Sequential, Flatten, Unflatten, ModuleList, Sigmoid, BatchNorm1d

def normalize(adj):
    inv_sqrt_sum = torch.pow(torch.sum(adj, axis=1), -0.5)
    inv_sqrt_sum[torch.isinf(inv_sqrt_sum)] = 0
    D_hat_inv_sqrt = torch.diag(inv_sqrt_sum)
    adj_norm = D_hat_inv_sqrt @ adj @ D_hat_inv_sqrt
    return adj_norm


def unpad(image, pad_width=26):
    return image[pad_width:-pad_width, pad_width:-pad_width]


def pad(image, pad_width=26):
    padded= torch.nn.functional.pad(image, (pad_width, pad_width, pad_width, pad_width), 'constant', 0)
    padded[torch.eye(padded.shape[0]).bool()] = 1
    return padded


class Pool(nn.Module):
    def __init__(self, num_features, top_k):
        super(Pool, self).__init__()
        self.proj = Linear(num_features, 1) 
        self.activation = Sigmoid()
        self.top_k = top_k
        
    def forward(self, x, adj):
        scores = self.activation(self.proj(x).squeeze())
        weights, idx = torch.topk(scores, self.top_k)
        H_top_k = x[idx][:, idx]
        adj_top_k = adj[idx][:, idx]
        return H_top_k, adj_top_k, idx


class GINConv(nn.Module):
    def __init__(self, mlp):
        super(GINConv, self).__init__()
        self.mlp = mlp 
        self.eps = nn.Parameter(torch.zeros(1), requires_grad=True)

    def forward(self, X, adj):
        X = self.mlp((1+self.eps) * X + adj @ X)
        
        return X

    
class GSR(nn.Module):
    def __init__(self, lr_dim=160, hr_dim=320):
        super(GSR, self).__init__()
        self.hr_to_lr_ratio = hr_dim // lr_dim
        self.hr_dim = hr_dim
        self.lr_dim = lr_dim
        init_range = torch.sqrt(torch.tensor(6.0) / (self.hr_dim + self.hr_dim))
        self.proj = nn.Parameter(torch.nn.init.uniform_(torch.empty(self.hr_dim, self.hr_dim), -init_range, init_range), requires_grad=True)
    
    def forward(self, x, adj): 
        _, eigen_vectors_lr = torch.linalg.eigh(adj, UPLO='U')
        Sd = torch.cat([eigen_vectors_lr for _ in range(self.hr_to_lr_ratio)], dim=0)
        
        A_hr = self.proj @ Sd @ x
        A_hr = torch.abs(A_hr)
        A_hr[torch.eye(A_hr.shape[0]).bool()] = 0
        A_hr = normalize(A_hr)
        
        H = A_hr @ A_hr.T
        H = (H + H.T) / 2
        # set diagonal to 1 
        H[torch.eye(H.shape[0]).bool()] = 1
        H = torch.abs(H)
        
        return H, A_hr  


class MLP(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.2, hidden_dim=320):
        super(MLP, self).__init__()
        self.proj1 = Linear(in_features, hidden_dim)
        self.proj2 = Linear(hidden_dim, out_features)
        self.activation = ReLU()
        self.bn1 = BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.proj1(x)
        x = self.activation(x)
        x = self.proj2(x)
        # x = self.activation(x)
        return x



class GraphSuperResolutionNet(nn.Module):
    def __init__(self, num_encoders=1, num_decoders=2):
        super(GraphSuperResolutionNet, self).__init__()
        # GCN Layers for feature extraction
        self.encoder = ModuleList([GINConv(MLP(160, 320))] + [GINConv(MLP(320, 320)) for _ in range(num_encoders-1)])
        self.super_res_layer = GSR()
        self.decoder = ModuleList([GINConv(MLP(268, 268)) for _ in range(num_decoders)])
        self.pool = Pool(320, 268)

    def forward(self, weighted_edges):
        H = torch.eye(160).float().to(weighted_edges.device)
        weighted_edges = normalize(weighted_edges)
        
        for layer in self.encoder:
            H = layer(H, weighted_edges)
        H, A_hr = self.super_res_layer(H, weighted_edges)
        
        H, A_hr, idx = self.pool(H, A_hr)
        for layer in self.decoder:
            H = layer(H, A_hr)
        
        H = (H + H.T) / 2
        # set diagonal to 1
        H[torch.eye(H.shape[0]).bool()] = 0
        H = torch.abs(H)
        
        return H, idx

In [ ]:
# Model
def get_model():
  model = GraphSuperResolutionNet(2, 2)
  model.to(device)
  return model

# Training

In [ ]:
def evaluate(pred, target):
    criterion = nn.L1Loss()
    # mask the diagonal and lower triangular part of the matrix
    mask = torch.triu(torch.ones_like(target, dtype=torch.bool), diagonal=1)
    loss = criterion(pred[mask], target[mask])
    return loss

def calculate_gsr_loss(lr, hr, model):
    hr_pred, idx = model(lr)
    criterion = nn.MSELoss()

    # hr_padded = pad(hr)
    _, U_hr = torch.linalg.eigh(hr, UPLO='U')

    loss = criterion(model.super_res_layer.proj[idx][:, idx], U_hr) + criterion(hr_pred, hr)

    return loss

# Hyperparameters
optimizer_fn = optim.Adam
epochs = 1000 # initial epochs to be changed
lr = 0.00038638729584238843

# 3-fold cross-validation using KFold
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=random_seed)
scores, all_metrics = k_fold_cross_validation(get_model, calculate_gsr_loss, evaluate, optimizer_fn, epochs, lr, step_size=15, gamma=0.993088589189151, training_only=True)
print("Mean validation loss:", np.mean(scores))

# Evaluation

In [ ]:
# generate the submission file
fold1Pred = getPred(1)

In [ ]:
def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")
    adj_matrix = np.array(adj_matrix)
    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict


def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path='ID-randomCV.csv'):
    print(true_hr_matrices.shape)

    # print(predicted_hr_matrices.shape)
    
    num_subjects = true_hr_matrices.shape[0]
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i]
        pred_matrix = predicted_hr_matrices[i]
        print("true matrix", true_matrix.shape)
        print("pred", pred_matrix.shape)

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")


In [ ]:
x1 = pd.read_csv('./fold1CSVX-Cluster.csv').values
x2 = pd.read_csv('./fold2CSVX-Cluster.csv').values
x3 = pd.read_csv('./fold3CSVX-Cluster.csv').values
comb1 = np.concatenate((x1,x2), axis=0)
final = np.concatenate((comb1, x3), axis=0)

df = pd.DataFrame(final)
output_path = 'clusterCV.csv'
if not df.empty:
    # Check if the file exists to decide whether to write headers
    file_exists = os.path.isfile(output_path)

    df.to_csv(output_path, mode='a', header=not file_exists, index=False)
    print(f"Results appended to {output_path}.")
else:
        print("No data to save.")

means = np.mean(final, axis=0)
std_devs = np.std(final, axis=0)

print(means)
print(std_devs)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt



def plot(data):
    avg_data = {metric: np.mean(values) for metric, values in data.items()}
    std_data = {metric: np.std(values) for metric, values in data.items()}

    fig, axs = plt.subplots(2, 2, figsize=(8, 8), sharey=True)
    colors = ['orange', 'green', 'steelblue', 'yellow', 'lightblue', 'lightgreen']
    ind = np.arange(len(data))
    width = 0.7

    for i in range(3):
        axs[i//2, i%2].bar(ind, [data[metric][i] for metric in data], width, color=colors)

    ax_avg = axs[1, 1]

    # Adding the average data with error bars to the last subplot
    ax_avg.bar(ind, list(avg_data.values()), width, color=colors, yerr=list(std_data.values()), capsize=5)

    # Adding some text for labels and custom x-axis tick labels, and set titles
    for i, ax in enumerate(axs.flat):
        if i < 3:  # For individual fold axes
            ax.set_title(f'Fold {i+1}')
        else:  # For the average plot
            ax.set_title('Avg. Across Folds')
        ax.set_xticks(ind)
        ax.set_xticklabels(data.keys())
        # ax.set_ylim(left=0)
        for tick in ax.get_xticklabels():
            tick.set_rotation(45)

    # Remove the empty subplot (if any)
    for i in range(len(data), 3):
        fig.delaxes(axs.flat[i])

    # Set the y-axis label
    axs[0, 0].set_ylabel('Metric Value')

    # Tight layout to adjust subplots automatically
    plt.tight_layout()

    # Display the plot
    plt.show()

data = {}
data['MAE'] = all_metrics['mae']
data['PCC'] = all_metrics['pcc']
data['JSD'] = all_metrics['jsd']

plot(data)

In [ ]:
data = {}
data['MAE (PC)'] = all_metrics['avg_mae_pc']
data['MAE (EC)'] = all_metrics['avg_mae_ec']
data['MAE (BC)'] = all_metrics['avg_mae_bc']

plot(data)

In [ ]:
ram_mean = np.mean(all_metrics['ram_usage'])
ram_std = np.std(all_metrics['ram_usage'])
time_mean = np.mean(all_metrics['time'])
time_std = np.std(all_metrics['time'])

print(f'Average RAM usage: {ram_mean} ± {ram_std} MB')
print(f'Average time: {time_mean} ± {time_std} seconds')